In [1]:
import numpy as np
import torch
from torch import nn
import torchvision

In [3]:
MNIST_dataset = torchvision.datasets.ImageNet('path/to/imagenet_root/')
data_loader = torch.utils.data.DataLoader(MNIST_dataset,
                                          batch_size=32,
                                          shuffle=True,)

RuntimeError: ignored

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [6]:
BATCH_SIZE=32

In [7]:
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
train_dataloader.dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [9]:
train_dataloader.dataset.class_to_idx

{'T-shirt/top': 0,
 'Trouser': 1,
 'Pullover': 2,
 'Dress': 3,
 'Coat': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'Bag': 8,
 'Ankle boot': 9}

In [10]:
for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]) torch.int64


In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
class MNISTNeuralNetwork(nn.Module):
  def __init__(self):
      super().__init__()
      self.flatten = nn.Flatten()
      self.linear_relu_stack = nn.Sequential(
          nn.Linear(28*28, 512),
          nn.ReLU(),
          nn.Linear(512, 256),
          nn.ReLU(),
          nn.Linear(256, 512),
          nn.ReLU(),
          nn.Linear(512, 10)
      )

  def forward(self, x):
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits

In [43]:
model_v0 = MNISTNeuralNetwork()
model_v0

MNISTNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [44]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_v0.parameters(), lr=0.0005)

In [45]:
model_v0.to(device)

MNISTNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [31]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import wandb

In [33]:
!wandb login

wandb: Currently logged in as: kevinbuhler. Use `wandb login --relogin` to force relogin


In [34]:
wandb.init(project="my-MNIST-project")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100,
  "batch_size": 32
}

test_loss,█▇▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,0.46443


In [35]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [36]:
def test(dataloader, model, loss_fn):
    accs = []
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    accs.append(test_loss)
    correct /= size
    wandb.log({"test_loss": test_loss})
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    


In [40]:
import os
import time

In [46]:
 timestamp = int(time.time())

In [48]:
epochs = 100
for epoch in range(epochs):
  train(train_dataloader, model_v0, loss_fn, optimizer)
  test(test_dataloader, model_v0, loss_fn)

  if epoch % 10:
    fn = f"models/tinyvoice_{timestamp}_{epoch}_mv0.pt"
    torch.save(model_v0.state_dict(), fn)
    print(f"saved model {fn} with size {os.path.getsize(fn)}")
  


Test Error: 
 Accuracy: 39.9%, Avg loss: 2.082162 

Test Error: 
 Accuracy: 49.0%, Avg loss: 1.808993 

saved model models/tinyvoice_1673164786_1_mv0.pt with size 2683213
Test Error: 
 Accuracy: 51.1%, Avg loss: 1.468807 

saved model models/tinyvoice_1673164786_2_mv0.pt with size 2683213
Test Error: 
 Accuracy: 59.2%, Avg loss: 1.246942 

saved model models/tinyvoice_1673164786_3_mv0.pt with size 2683213
Test Error: 
 Accuracy: 60.8%, Avg loss: 1.110160 

saved model models/tinyvoice_1673164786_4_mv0.pt with size 2683213
Test Error: 
 Accuracy: 63.3%, Avg loss: 1.011874 

saved model models/tinyvoice_1673164786_5_mv0.pt with size 2683213
Test Error: 
 Accuracy: 65.3%, Avg loss: 0.939214 

saved model models/tinyvoice_1673164786_6_mv0.pt with size 2683213
Test Error: 
 Accuracy: 67.2%, Avg loss: 0.883103 

saved model models/tinyvoice_1673164786_7_mv0.pt with size 2683213
Test Error: 
 Accuracy: 68.8%, Avg loss: 0.840411 

saved model models/tinyvoice_1673164786_8_mv0.pt with size 2683

KeyboardInterrupt: ignored